In [1]:
import pandas as pd
import ast
import numpy as np

Transformación de Movies

In [ ]:
df_movies = pd.read_csv("C:/Users/jano_/OneDrive/Escritorio/Proyecto Final Henry/Movies/movies_dataset.csv") #apertura csv movie_dataset.csv

In [90]:
#eliminación de columnas que no serán necesarias para desarrollar el modelo

df_movies.drop(["video", "imdb_id", "adult", "original_title", "homepage", "poster_path", "status", "spoken_languages", "production_countries"], axis=1, inplace=True) 

In [91]:
""" Se eliminan filas donde en release_date hayan nulos ya que será un parámetro fundamental para desarrollar las funciones de la API, 
además se realiza lo mismo sobre la columna popularity ya que será un argumento de recorte del dataset."""

df_movies.dropna(subset= ["release_date"], inplace= True)
df_movies.dropna(subset= ["popularity"], inplace= True)

In [92]:
# Recorte del dataset según la popularidad de las peliculas que contiene el mismo

popularity_min = 8.0
df_movies["popularity"] = pd.to_numeric(df_movies["popularity"], errors= "coerce")
df_movies = df_movies[df_movies["popularity"] >= popularity_min]

In [93]:
""" A continuación tomamos los valores de belongs_to_collections cómo diccionarios(para esto implementamos la librería ast ya que son datos tipo string) 
 y desanidamos los valores en un nueva tabla para colecciones, a su vez convertimos los valores de id en enteros. """

df_movies["belongs_to_collection"] = df_movies["belongs_to_collection"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

df_collection = df_movies["belongs_to_collection"].apply(lambda x: {
    "idCollection": x["id"],
    "nameCollection": x["name"]
}
if isinstance(x, dict) else None).apply(pd.Series)

df_collection['idCollection'] = df_collection['idCollection'].astype('Int64')

In [94]:
# Normalizamos id de df_movies y a este mismo dataframe le concatenamos idCollections para consultas.
df_movies = df_movies.rename(columns={"id" : "idMovies"})
df_movies['idMovies'] = df_movies['idMovies'].astype(int)
df_movies = pd.concat([df_movies , df_collection["idCollection"]], axis=1)

In [95]:
# Se eliminan duplicados de df_collections, ya fueron copiados los idCollection en cada pelicula de df_movies y filas con valores nulos
df_collection = df_collection.drop_duplicates(subset='idCollection', keep='first')
df_collection.dropna(inplace=True)

In [96]:
df_collection.to_parquet("Collections.parquet", engine= "pyarrow", index= False)

In [97]:
""" Se desanida genres de df_movies con función explode, 
que separará cada elemento de las listas contenidas en la columna genres y duplicará en filas por cada elemento."""

df_movies["genres"] = df_movies["genres"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df_aux_genres = df_movies.explode("genres").reset_index(drop=True)

In [98]:
# Generamos la tabla df_genres, que contendra id y nombre de los géneros.

df_genres= df_aux_genres["genres"].apply(lambda x: {
    "idGenres": x["id"],
    "nameGenres": x["name"]
}
if isinstance(x, dict) else None).apply(pd.Series)

df_genres["idGenres"] = df_genres["idGenres"].astype("Int64")

In [99]:
# Generamos una tabla complementaria para poder relacionar la entidad Género y Movie del modelo.

df_genres_movies = pd.concat([df_genres["idGenres"], df_aux_genres["idMovies"]], axis= 1)
del df_aux_genres
df_genres_movies.to_parquet("DataSet/Relation_Movies_Genres.parquet", engine= "pyarrow", index= False)

In [100]:
# Se eliminan duplicados de la tabla df_genres y guardamos el data frame cómo un archivo parquet.

df_genres = df_genres.drop_duplicates(subset='idGenres', keep='first')
df_genres.to_parquet("Genres.parquet", engine="pyarrow", index= False)

In [101]:
""" Se desanida production_companies de df_movies con función explode, 
que separará cada elemento de las listas contenidas en la columna production_companies y duplicará en filas por cada elemento."""

df_movies["production_companies"] = df_movies["production_companies"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df_aux_companies = df_movies.explode("production_companies").reset_index(drop=True)

In [102]:
# Generamos la tabla df_prod_companies, que contendra id y nombre de las compañías productoras.

df_prod_companies= df_aux_companies["production_companies"].apply(lambda x: {
    "idCompany": x["id"],
    "nameCompany": x["name"]
}
if isinstance(x, dict) else None).apply(pd.Series)

In [103]:
# Generamos una tabla complementaria para poder relacionar la entidad Production_companies y Movies del modelo.

df_prod_company_movies = pd.concat([df_prod_companies["idCompany"], df_aux_companies["idMovies"]], axis= 1)
del df_aux_companies
df_prod_company_movies.dropna(inplace=True)
df_prod_company_movies.to_parquet("DataSet/Relation_Movies_Companies.parquet", engine= "pyarrow", index= False)

In [104]:
# Se eliminan duplicados y valorez nulos de la tabla df_prod_companies y guardamos el data frame cómo un archivo parquet.

df_prod_companies.dropna(inplace=True)
df_prod_companies = df_prod_companies.drop_duplicates(subset='idCompany', keep='first')
df_prod_companies.to_parquet("Production_companies.parquet", engine="pyarrow", index= False)

In [123]:
# Eliminamos las columnas desanidadas.
df_movies.drop(columns= ["belongs_to_collection"], inplace= True)
#df_movies.drop(columns= ["genres","production_companies", "belongs_to_collection"], inplace= True)

In [109]:
# Reemplazamos valores nulos por 0 en las columnas revenue y budget, a su vez creamos columna con el retorno dividiendo revenue por budget,
# siempre que ninguno de las dos contenga 0.
df_movies["revenue"] = pd.to_numeric(df_movies["revenue"], errors="coerce")
df_movies["budget"] = pd.to_numeric(df_movies["budget"], errors="coerce")
df_movies [["revenue", "budget"]] = df_movies [["revenue", "budget"]].fillna(0)
df_movies["return"] = np.where(
    (df_movies["revenue"] != 0) & (df_movies["budget"] != 0),
    df_movies["revenue"] / df_movies["budget"],
    0
)

In [110]:
# Cambiamos el formato a la columna release_date y agregamos una columna nueva donde extraemos el año de estreno para luego usarla en el modelo de ML.

df_movies["release_date"] = pd.to_datetime(df_movies["release_date"], format= "%Y-%m-%d")
df_movies["release_year"] = df_movies ["release_date"].dt.year

In [124]:
df_movies.to_parquet("Movies.parquet", engine= "pyarrow", index= False)

Transformación de Credits

In [112]:
df_credits = pd.read_csv("C:/Users/jano_/OneDrive/Escritorio/Proyecto Final Henry/Movies/credits.csv")

In [113]:
# Se realiza recorte del dataset según las películas que recortamos en movie, por popolarity > 8.0
df_credits = df_credits.rename(columns={"id" : "idMovies"})
df_credits_short = df_credits[df_credits["idMovies"].isin(df_movies["idMovies"])]

In [ ]:
"""La primer línea del código toma los valores como listas y/o diccionario de python para poder luego desanidar con explode,
 a su vez recortamos la cantidad de actores según el orden en que aparecen primero ya que se observó que los actores principales son los primeros
 en aparecer."""
 
df_credits_short["cast"] = df_credits_short["cast"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df_cast_aux = df_credits_short.explode("cast").reset_index(drop=True)
df_cast_aux = df_cast_aux.groupby("idMovies").head(15) 

In [115]:
df_cast= df_cast_aux["cast"].apply(lambda x: {
    "idCast": x["id"],
    "nameCast": x["name"]
}
if isinstance(x, dict) else None).apply(pd.Series)

df_cast["idCast"] = df_cast["idCast"].astype("Int64")

In [116]:
df_cast_movies = pd.concat([df_cast["idCast"], df_cast_aux["idMovies"]], axis= 1)
df_cast_movies.to_parquet("DataSet/Relation_Cast_Movies.parquet", engine= "pyarrow", index= False)

In [117]:
df_cast = df_cast.drop_duplicates(subset='idCast', keep='first')
df_cast.to_parquet("DataSet/Cast_Movies.parquet", engine= "pyarrow", index= False)

In [ ]:
df_credits_short["crew"] = df_credits_short["crew"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df_crew_aux = df_credits_short.explode("crew").reset_index(drop=True)

In [119]:
df_crew= df_crew_aux["crew"].apply(lambda x: {
    "idCrew": x["id"],
    "nameCrew": x["name"],
    "nameJob": x["job"],
}
if isinstance(x, dict) else None).apply(pd.Series)


In [120]:
df_crew["idCrew"] = df_crew["idCrew"].astype("Int64")
df_crew = pd.concat([df_crew , df_crew_aux["idMovies"]], axis=1)

In [121]:
df_director_aux = df_crew[df_crew["nameJob"] == "Director"]
df_movies_director = df_director_aux[["idMovies", "idCrew"]]
df_movies_director.to_parquet("DataSet/Relation_Director_Movies.parquet", engine= "pyarrow", index= False)


In [122]:
df_director = df_director_aux[["idCrew", "nameCrew"]].drop_duplicates().reset_index(drop=True)
df_director.to_parquet("DataSet/Director_Movies.parquet", engine= "pyarrow", index= False)